## mindspore.ops.std(input, axis=None, ddof=0, keepdims=False) -〉Tensor
按指定维度计算标准差。
- 输入：
    * input: mindspore的tensor, 数据类型为number.Number。
    * axis: int, tuple(int)。
    * ddof: int, bool。
    * keepdims: bool。

- 返回：mindspore的tensor。

1、参数比较：
| mindspore   | torch       | jax         |
| :----:      | :----:      | :----:      |
| input       | input       | a           |
| axis        | dim         | axis        |
| ddof        | correction  | dtype       |
| keepdims    | keepdim     | ddof        |
|             | out         | keepdims    |
|             |             | where       |
|             |             | correction  |
|             |             | out (unused) |

* torch提供了out出参方式。
* jax额外提供了where参数，表示是否要参与计算标准差，dtype用于设置返回值的数据类型。correction与ddof相同作用，不能同时配置。

2、返回值比较

In [5]:
import numpy as np
import mindspore as ms
import torch
import jax.numpy as jnp

input = np.array([[1., 3, 4, 2],
                  [5, 2, 6, 3],
                  [8, 1, 3, 9]])

y1 = ms.ops.std(ms.tensor(input))
y2 = torch.std(torch.tensor(input))
y3 = jnp.std(input)
print ('mindspore output:',y1)
print ('torch output:',y2)
print ('jax outputn',y3)

mindspore output: 2.498610734939575
torch output: tensor(2.6097, dtype=torch.float64)
jax outputn 2.4986107


* CPU下实测发现ms与torch不支持int作为输入，jax支持。
* m与jax都不返回类型。

In [9]:
y1 = ms.ops.std(ms.tensor(input), 1, keepdims=True)
y2 = torch.std(torch.tensor(input), 1, keepdim=True)
y3 = jnp.std(input, 1, keepdims=True)
print ('mindspore output:\n',y1)
print('\n')
print ('torch output:\n',y2)
print('\n')
print ('jax output:\n',y3)

mindspore output:
 [[1.11803401]
 [1.58113885]
 [3.3447721 ]]


torch output:
 tensor([[1.2910],
        [1.8257],
        [3.8622]], dtype=torch.float64)


jax output:
 [[1.118034 ]
 [1.5811388]
 [3.344772 ]]


使用where，来确定计算范围时：

In [10]:
where=np.array([[0, 0, 0, 0],
                [0, 1, 0, 1],
                [1, 1, 1, 0]], dtype=bool)
y3 = jnp.std(input, 1, keepdims=True, where=where)
print ('jax output:\n',y3)

jax output:
 [[      nan]
 [0.5      ]
 [2.9439204]]


3、报错信息比较

In [11]:
y1 = ms.ops.std(input, 1)

TypeError: For std, input must be a Tensor, but got <class 'numpy.ndarray'>.

In [12]:
y2 = torch.std(input, 1)

TypeError: std() received an invalid combination of arguments - got (numpy.ndarray, int), but expected one of:
 * (Tensor input, tuple of ints dim, bool unbiased, bool keepdim, *, Tensor out)
 * (Tensor input, tuple of ints dim, *, Number correction, bool keepdim, Tensor out)
 * (Tensor input, bool unbiased)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !int!)
 * (Tensor input, tuple of names dim, bool unbiased, bool keepdim, *, Tensor out)
 * (Tensor input, tuple of names dim, *, Number correction, bool keepdim, Tensor out)


In [13]:
input = [2, 4, 3, 1]
y3 = jnp.std(input, 1)

TypeError: std requires ndarray or scalar arguments, got <class 'list'> at position 0.

当输入类型不正确时，报错信息torch简洁明确。建议ms优化。